In [ ]:
import os
import time
import config
from datetime import datetime
from log_utils import read_log_messages, normalize_template, normalize_template_v1, normalize_template_v2, save_templates
from ollama_utils import start_ollama_server, stop_ollama_server, start_ollama_server_log
from autogen import AssistantAgent
from codecarbon import EmissionsTracker, OfflineEmissionsTracker
from evaluation import evaluate_and_save

In [ ]:
# --- Configuration ---
llm_config = config.LLM_CONFIG

DATA_DIR = config.DATA_DIR
RESULT_DIR = config.RESULT_DIR
os.makedirs(RESULT_DIR, exist_ok=True)
 
TASK = (config.TASK).capitalize()
DESIGN = (config.DESIGN).capitalize()
project_name = f"{TASK}_{DESIGN}"
model = llm_config["config_list"][0]["model"].replace(":", "-")
timestamp = datetime.now().strftime("%Y%m%d-%H%M%S")
exp_name = f"{project_name}_{model}_{timestamp}"

input_file = config.IN_FILE
input_file_path = os.path.join(DATA_DIR, input_file)
ground_truth_file = config.GT_FILE
ground_truth_file_path = os.path.join(DATA_DIR, ground_truth_file)



In [ ]:
# --- Task-specific configuration ---
task_prompt = None
sys_prompt_generator = None

# Normalize lowercase for comparisons
task_lower = TASK.lower()
design_lower = DESIGN.lower()

if task_lower == "log-parsing":
    task_prompt = config.TASK_PROMPT_LOG_PARSING
    if design_lower == "sa-few":
        sys_prompt_generator = config.SYS_MSG_SINGLE_LOG_PARSER_GENERATOR_FEW_SHOT
    elif design_lower == "sa-zero":
        sys_prompt_generator = config.SYS_MSG_SINGLE_LOG_PARSER_GENERATOR_ZERO_SHOT
    elif design_lower == "da-few":
        sys_prompt_generator = config.SYS_MSG_SINGLE_LOG_PARSER_GENERATOR_FEW_SHOT
        sys_prompt_critic = config.SYS_MSG_SINGLE_LOG_PARSER_CRITIC_FEW_SHOT
    elif design_lower == "da-zero":
        sys_prompt_generator = config.SYS_MSG_SINGLE_LOG_PARSER_GENERATOR_ZERO_SHOT
        sys_prompt_critic = config.SYS_MSG_SINGLE_LOG_PARSER_CRITIC_ZERO_SHOT
    elif design_lower == "ma-few":
        sys_prompt_generator = config.SYS_MSG_SINGLE_LOG_PARSER_GENERATOR_FEW_SHOT
        sys_prompt_critic = config.SYS_MSG_SINGLE_LOG_PARSER_CRITIC_FEW_SHOT
        sys_prompt_refiner = config.SYS_MSG_SINGLE_LOG_PARSER_REFINER_FEW_SHOT
        sys_prompt_proxy = config.SYS_MSG_SINGLE_LOG_PARSER_PROXY_FEW_SHOT
    elif design_lower == "ma-zero":
        sys_prompt_generator = config.SYS_MSG_SINGLE_LOG_PARSER_GENERATOR_ZERO_SHOT
        sys_prompt_critic = config.SYS_MSG_SINGLE_LOG_PARSER_CRITIC_ZERO_SHOT
        sys_prompt_refiner = config.SYS_MSG_SINGLE_LOG_PARSER_REFINER_ZERO_SHOT
        sys_prompt_proxy = config.SYS_MSG_SINGLE_LOG_PARSER_PROXY_ZERO_SHOT
    else:
        raise ValueError(f"Unsupported DESIGN: {DESIGN}")

elif task_lower == "td-detection":
    task_prompt = config.TASK_PROMPT_TD_DETECTION
    if design_lower == "ma-zero":
        sys_prompt_generator = config.SYS_MSG_TD_DETECTION_GENERATOR_ZERO_SHOT
        sys_prompt_critic = config.SYS_MSG_TD_DETECTION_CRITIC_ZERO_SHOT
        sys_prompt_refiner = config.SYS_MSG_TD_DETECTION_REFINER_ZERO_SHOT
        sys_prompt_proxy = config.SYS_MSG_SINGLE_TD_DETECTION_PROXY_ZERO_SHOT

elif task_lower == "log-analysis":
    # Assign log-analysis-specific prompts here
    pass

elif task_lower == "code-generation":
    # Assign code-generation-specific prompts here
    pass

elif task_lower == "vul-detection":
    task_prompt = config.VULNERABILITY_TASK_PROMPT
    if design_lower == "ma-zero":
        

else:
    raise ValueError(f"Unsupported TASK: {TASK}")


In [ ]:
# --- Log Reading ---
logs = read_log_messages(input_file_path)

In [ ]:
# --- With CodeCarbon Emissions Tracking for Log Parsing Task---
def run_inference_with_emissions_log_parsing(logs, llm_config, sys_prompt_log_parser, task_prompt, exp_name, result_dir):
    parsed_templates = []
    tracker = OfflineEmissionsTracker(project_name=exp_name, output_dir=result_dir, country_iso_code="CAN", save_to_file=True)
    tracker.start()
    try:
        log_parser = create_agent(
            agent_type="assistant",
            name="log_parser_agent",
            llm_config=llm_config,
            sys_prompt=sys_prompt_log_parser,
            description="Analyze the log message in order to determine the corresponding template."
        )
        for i, log_message in enumerate(logs):
            content = task_prompt + log_message
            res = log_parser.generate_reply(messages=[{"content": content, "role": "user"}])
            if res is not None and "content" in res:
                parsed_templates.append(res["content"].strip())
            else:
                parsed_templates.append("NONE")
                print(f"[Warning] Skipped log {i} — no response or invalid format.")
    finally:
        emissions = tracker.stop()
    print(f"Emissions: {emissions} kg CO2")
    return parsed_templates


In [ ]:
# --- Main Execution ---
#proc = start_ollama_server()
time.sleep(5)  # Give it some time to initialize
parsed_templates = run_inference_with_emissions_log_parsing(logs, llm_config, sys_prompt_few_shot_single_log_parser, task_prompt, exp_name, RESULT_DIR)
save_templates(parsed_templates, llm_config, DESIGN, RESULT_DIR)
#stop_ollama_server(proc)

In [ ]:
# --- Agent Creation ---
def create_single_log_parser_agent(llm_config, sys_prompt):
    return AssistantAgent(
        name="log_parser_agent",
        system_message=sys_prompt,
        description="Analyze the log message in order to determine the corresponding template.",
        llm_config=llm_config,
        human_input_mode="NEVER",
    )

In [ ]:
# --- Log Reading ---
logs = read_log_messages(log_path)

In [ ]:
# --- Print Results ---
#print("Templates:", parsed_templates)

In [ ]:
results = evaluate_and_save(normalize_template, parsed_templates, ground_truth_file_path, exp_name)
results_v1 = evaluate_and_save(normalize_template_v1, parsed_templates, ground_truth_file_path, exp_name)
results_v2 = evaluate_and_save(normalize_template_v2, parsed_templates, ground_truth_file_path, exp_name)

In [ ]:
#print("Results:", results)